# Heuristic Core Greedy Algorithms

## Application of core greedy algorithms for general heuristic problem-solving using Python

    Original publish date: May, 2023
    Version: v1
    License: MIT
    Author: Alejandro Asensio Pérez
    Tags: Heuristic, Greedy Algorithms, Python

### Summary

The notebook explores the fundamental concepts of greedy algorithms and their application in heuristic problem-solving. It provides a collection of interactive Python notebooks.

### Connect and Engage

Please, feel free to comment on typos, propose improvements or expand the content.

In [1]:
# Import standard libraries
from math import inf

## Dijkstra's Algorithm

Shortest Path problem-solving for undirected graphs

In [2]:
# Function to prepare adjacency list
def prepare_adjacency_list(n, e):
  graph = [[] for _ in range(n + 1)]
  for u, v, w in e:
    graph[u].append((v, w))
  return graph


def dijkstra_sparse(graph, src):
  """
  Implements Dijkstra's algorithm to find the shortest paths from a starting node to all other nodes.

  This implementation uses an adjacency list representation of the graph.

  It is more efficient for sparse graphs where the number of edges is much less than the square of the number of nodes.

  The algorithm iterates over the neighbors of each node to update distances.

  Args:
    graph: An adjacency list representation of the graph, where each element is a list of tuples (neighbor, weight).
    src: The index of the starting node.

  Returns:
    A tuple containing two lists:
        - distances: A list of shortest distances from the starting node to all other nodes.
        - parent: A list of parent node indices for the shortest paths.
  """

  def find_min_unvisited_node(distances, visited):
    """
    Finds the unvisited node with the minimum distance in the graph.

    Args:
        distances: A list of distances for each node in the graph.
        visited: A list of booleans indicating whether a node has been visited.

    Returns:
        The index of the unvisited node with the minimum distance, or None if all nodes are visited.
    """

    min_dist = inf
    min_index = 0

    for i in range(1, len(distances)):
      if not visited[i] and distances[i] < min_dist:
        min_dist = distances[i]
        min_index = i

    return min_index

  n = len(graph)  # Num nodes

  dist = [inf] * n
  dist[src] = 0  # Set starting node distance to 0

  path = [-1] * n  # Initialize the shortest path list

  visited = [False] * n
  visited[src] = True  # Mark starting node as visited

  # Initialize distances for neighbors of the starting node
  for dest, weight in graph[src]:
    dist[dest] = weight
    path[dest] = src

  # Main loop for Dijkstra's algorithm
  for _ in range(2, n):
    # Find the unvisited node with the minimum distance
    neighbor = find_min_unvisited_node(dist, visited)

    visited[neighbor] = True

    # Relax edges from the current node
    for dest, weight in graph[neighbor]:
      new_dist = dist[neighbor] + weight
      if dist[dest] > new_dist:
        dist[dest] = new_dist
        path[dest] = neighbor

    return dist, path

In [3]:
# Function to prepare adjacency matrix
def prepare_adjacency_matrix(n, e):
  graph = [[inf] * (n + 1) for _ in range(n + 1)]
  for i in range(n + 1):
    graph[i][i] = 0
  for u, v, w in e:
    graph[u][v] = w
    graph[v][u] = w
  return graph


def dijkstra_dense(graph, src):
  """
  Implements Dijkstra's algorithm to find the shortest paths from a starting node to all other nodes.

  This implementation uses an adjacency matrix representation of the graph.

  It is more suitable for dense graphs where the number of edges is close to the square of the number of nodes.

  The algorithm maintains a list of candidate nodes and updates distances by selecting the closest candidate.

  Args:
      graph: An adjacency matrix representation of the graph.
      src: The index of the starting node.

  Returns:
      A tuple containing two lists:
          - distances: A list of shortest distances from the starting node to all other nodes.
          - parent: A list of parent node indices for the shortest paths.
  """

  def select_closest_candidate(candidates):
    """
    Selects the candidate with the minimum distance from the list.

    Args:
        candidates: A list of tuples containing (neighbor_index, distance) for candidate nodes.

    Returns:
        A tuple containing (neighbor_index, distance) for the closest candidate.
    """

    # Find the index of the candidate with the minimum distance
    min_index = 0
    for i in range(1, len(candidates)):
      if candidates[i][1] < candidates[min_index][1]:
        min_index = i

    # Return a copy of the closest candidate
    closest_candidate = candidates[min_index].copy()
    del candidates[min_index]

    return closest_candidate

  def update_dist_and_path(distances, candidate, graph, candidates, path):
    """
    Updates distances and parent nodes in the graph based on the selected closest candidate.

    Args:
        distances: A list of distances for each node in the graph.
        candidate: A tuple containing (neighbor_index, distance) for the closest candidate.
        graph: An adjacency list representation of the graph, where each element is a list of tuples (neighbor, weight).
        candidates: A list of tuples containing (neighbor_index, distance) for remaining candidate nodes.
        path: A list of parent node indices for the shortest paths (initially all -1).

    Returns:
        A tuple containing the updated distances and parent lists.
    """

    # Update distance of the selected candidate
    distances[candidate[0]] = candidate[1]

    # Relax edges for remaining candidates
    for neighbor, current_dist in candidates:
      new_dist = candidate[1] + graph[candidate[0]][neighbor - 1]  # Assuming 0-based indexing
      if new_dist < current_dist:
        candidates[candidates.index((neighbor, current_dist))] = (neighbor, new_dist)  # Update candidate distance
        path[neighbor] = candidate[0]  # Update path of the neighbor

    return distances, path

  path = [-1] * len(graph)
  dist = graph[src][:]

  # Initializes a list of candidate nodes and their distances from the starting node
  candidates = [[i, graph[src][i]]
                for i in range(1, len(graph))
                if i != src]

  # Initializes the path to the starting node for all neighbors of the starting node
  for i in range(2, len(graph[1])):
    if graph[1][i] != inf:
      path[i] = 1

  # Main loop for Dijkstra's algorithm
  while candidates:
    candidate = select_closest_candidate(candidates)
    dist, path = update_dist_and_path(dist, candidate, graph, candidates, path)
    return dist, path

### Test case and Asserts

In [4]:
source = 1  # Starting node

n_nodes = 3  # Num nodes

edges = [
  (3, 1, 1),
  (3, 2, 2),
  (1, 3, 1),
  (2, 3, 1),
  (2, 1, 2),
  (1, 2, 2)
]

graph_adj_list = prepare_adjacency_list(n_nodes, edges)

graph_adj_matrix = prepare_adjacency_matrix(n_nodes, edges)

In [5]:
# Expected results
expected_distances = [inf, 0, 2, 1]
expected_paths = [-1, -1, 1, 1]

# Run Dijkstra's algorithm for sparse graph
distances, paths = dijkstra_sparse(graph_adj_list, source)
assert distances == expected_distances, "Problem with Dijkstra's algorithm (sparse)"
assert paths == expected_paths, "Problem with Dijkstra's algorithm (sparse)"

# Run Dijkstra's algorithm for dense graph
distances, paths = dijkstra_dense(graph_adj_matrix, source)
assert distances == expected_distances, "Problem with Dijkstra's algorithm (dense)"
assert paths == expected_paths, "Problem with Dijkstra's algorithm (dense)"

print("All tests passed.")

All tests passed.


In [6]:
%%timeit
dijkstra_sparse(graph_adj_list, source)

620 ns ± 4.45 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%%timeit
dijkstra_dense(graph_adj_matrix, source)

839 ns ± 5.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Kruskal's Algorithm

Minimum Spanning Tree problem-solving for undirected graphs

In [8]:
# Function to prepare graph
def prepare_graph(e):
  edges_list = []
  for u, v, e in e:
    edges_list.append((u, v, e))
  return sorted(edges_list, key=lambda edge: edge[2])


def kruskal(graph, n):
  """
  Implements Kruskal's algorithm to find the minimum spanning tree (MST) of an undirected graph.

  Args:
    graph: An adjacency list representation of the graph, where each element is a list of tuples (neighbor, weight).
    n: The total number of nodes in the graph.

  Returns:
    A list of edges representing the MST.
  """

  def update_connected_components(components, edge):
    """
    Updates connected component labels for nodes merged based on an edge.

    Args:
        components: A list representing connected component labels for each node.
        edge: A tuple containing the indices of two nodes connected by an edge.

    Returns:
        The updated list of connected component labels.
    """

    # Get the labels of the source and target nodes
    source_label = components[edge[0]]
    target_label = components[edge[1]]

    # Update all nodes with the source label to the target label
    for i in range(len(components)):
      if components[i] == source_label:
        components[i] = target_label

    return components

  # Initialize empty solution and connected component labels
  solution = []
  connected_components = list(range(n + 1))

  # Iterate until we have n-1 edges and there are still edges in the graph
  while len(solution) != n - 1 and graph:
    # Select the next edge with the smallest weight
    current_edge = graph.pop(0)

    # Check if the edge connects nodes from different connected components
    if connected_components[current_edge[0]] != connected_components[current_edge[1]]:
      # Add the edge to the solution and update connected components
      solution.append(current_edge)
      connected_components = update_connected_components(connected_components, current_edge)

  return solution

### Test case and Asserts

In [9]:
n_nodes = 21  # Num nodes

edges = [
  (1, 2, 1),
  (1, 3, 4),
  (1, 4, 3),
  (1, 5, 10),
  (2, 3, 2),
  (3, 4, 2),
  (3, 7, 8),
  (4, 5, 7),
  (4, 8, 6),
  (5, 9, 1),
  (2, 6, 5),
  (6, 7, 3),
  (6, 10, 9),
  (7, 8, 4),
  (8, 9, 2),
  (9, 10, 5),
  (2, 4, 6),
  (3, 5, 7),
  (6, 8, 4),
  (7, 9, 3)
]

graph = prepare_graph(edges)

In [10]:
# Expected result
expected_solution = [(1, 2, 1), (5, 9, 1), (2, 3, 2), (3, 4, 2), (8, 9, 2), (6, 7, 3), (7, 9, 3), (2, 6, 5), (9, 10, 5)]

# Run Kruskal's algorithm
mst = kruskal(graph, n_nodes)
assert mst == expected_solution, "Problem with Kruskal's algorithm"

print("All tests passed.")

All tests passed.


In [11]:
%%timeit
kruskal(graph, n_nodes)

249 ns ± 5.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Backtracking

### Knapsack Problem

Combinatorial optimization problem-solving

In [12]:
def knapsack(weights, values, capacity):
  """
  Solves the 0/1 knapsack problem using a recursive backtracking algorithm.

  Args:
      weights: A list of weights for each item.
      values: A list of values for each item.
      capacity: The maximum weight that the knapsack can hold.

  Returns:
      A tuple containing the maximum value that can be achieved and a list of selected items.
  """

  def knapsack_recursive(index, current_weight, current_value, selected_items):
    """
    Recursive function to explore all possible solutions to the knapsack problem.

    Args:
        index: The index of the current item.
        current_weight: The total weight of items selected so far.
        current_value: The total value of items selected so far.
        selected_items: A list of indices of items selected so far.

    Returns:
        A tuple containing the maximum value that can be achieved and a list of selected items.
    """

    nonlocal max_value, best_selection

    # Base case: if we have processed all items
    if index == len(weights):
      if current_value > max_value:
        max_value = current_value
        best_selection = selected_items.copy()
      return

    # Check if we can select the current item
    if current_weight + weights[index] <= capacity:
      # Select the current item
      selected_items.append(index)
      knapsack_recursive(index + 1, current_weight + weights[index], current_value + values[index], selected_items)
      selected_items.pop()

    # Skip the current item
    knapsack_recursive(index + 1, current_weight, current_value, selected_items)

  max_value = 0
  best_selection = []

  knapsack_recursive(0, 0, 0, [])

  return max_value, best_selection

#### Test case and Asserts

In [13]:
capacity = 50

weights = [5, 10, 15, 7, 8, 9, 4, 3, 6, 12]

values = [10, 40, 30, 20, 50, 60, 70, 80, 90, 100]

In [14]:
# Expected output
expected_value = 470
expected_selection = [3, 4, 5, 6, 7, 8, 9]

# Run the recursive backtracking algorithm
value, selection = knapsack(weights, values, capacity)
assert value == expected_value, "Problem with the knapsack algorithm"
assert selection == expected_selection, "Problem with the knapsack algorithm"

print("All tests passed.")

All tests passed.


In [15]:
%%timeit
knapsack(weights, values, capacity)

159 μs ± 513 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Maze Exploration

Automated method for solving a maze

In [16]:
def maze_exploration(maze, pos, n_objects, min_steps, visited, last_object=0, current_steps=0):
  """
  Recursive function to explore the maze and find the shortest path to collect all objects.

  Args:
      maze: A 2D list representing the maze.
      pos: A tuple containing the current position in the maze.
      n_objects: The total number of objects to collect.
      last_object: The index of the last object collected.
      min_steps: The current minimum number of steps to reach the goal.
      current_steps: The current number of steps taken.
      visited: A set containing the visited cells.

  Returns:
      The minimum number of steps to reach the goal.
  """

  # Check if we have collected all objects
  if last_object == n_objects:
    return min(min_steps, current_steps)

  # Explore all possible movements
  for x_dir, y_dir in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
    x, y = pos[0] + x_dir, pos[1] + y_dir

    is_within_bounds = 0 <= x < len(maze) and 0 <= y < len(maze[0])
    is_not_visited = (x, y) not in visited
    if is_within_bounds and is_not_visited:
      last_object_found = maze[x][y] == last_object + 1
      is_empty = maze[x][y] == 0
      if last_object_found or is_empty:
        visited.add((x, y))

        if last_object_found:
          last_object += 1

        min_steps = min(
          maze_exploration(maze, (x, y), n_objects, min_steps, visited, last_object, current_steps + 1),
          min_steps)

        if last_object_found:
          last_object -= 1

        visited.remove((x, y))

  return min_steps

#### Test case and Asserts

In [17]:
maze = [
  [0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 3, 0, 0, 0],
  [0, 0, 0, 0, 2]
]

init_cell = (0, 0)

objects = 3

visited = {init_cell}

In [18]:
# Expected output
expected_path = 12

# Run the backtracking algorithm
path = maze_exploration(maze, init_cell, objects, inf, visited)
assert path == expected_path, "Problem with the backtracking algorithm"
assert path != inf, "No path found"

print("All tests passed.")

All tests passed.


In [19]:
%%timeit
maze_exploration(maze, init_cell, objects, inf, visited)

20.8 ms ± 63.9 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Thanks for following along!** 🙂 If you have implemented a similar approach or have ideas for improvement, I'd love to see your code.

Also, if you find the work helpful and would like to cite them, you can use the following bibtex:

```bibtex
@misc{aaseper2024hcga,
   title        = {{Heuristic Core Greedy Algorithms}},
   author       = {Alejandro Asensio},
   year         = 2024,
   howpublished = {\url{https://drive.google.com/file/d/1IikuoaccPDaEJkQJlG6FS_93fxfNEJqG/view?usp=sharing}}
}
```

© Copyright 2024 Alejandro Asensio Pérez.